In [20]:
import requests
from pyquery import PyQuery as pq
import pandas as pd
import datetime as dt
import time

In [6]:
url = "https://lottery.olg.ca/en-ca/winning-numbers/lotto-max/winning-numbers?startDate=2018-11-22&endDate=2018-11-24"
#r = requests.get(url)
r = pq(url = url )

In [7]:
t = dt.date(2018, 11, 23)

In [8]:
(t - dt.timedelta(1)).strftime("%Y-%m-%d")

'2018-11-22'

In [9]:
def generateURL(t):
    s = (t - dt.timedelta(1)).strftime("%Y-%m-%d")
    e = (t + dt.timedelta(1)).strftime("%Y-%m-%d")
    return "https://lottery.olg.ca/en-ca/winning-numbers/lotto-max/winning-numbers?startDate=" + s + "&endDate="+ e


In [10]:
def process(t):
    url = generateURL(t)
    r = pq(url = url )
    wn, p, twc = parseResult(r)
    print(f"winning number={wn} prize = {p} total winners = {twc}")
    dic = {'date': t.strftime("%Y-%m-%d"), "wn1": "-".join([str(i) for i in wn]), 
           "prize":p[0], "7/7": twc[0], "6B/7": twc[1], "6/7":twc[2], "5/7":twc[3], "4/7":twc[4], "3B/7":twc[5],"3/7":twc[6]}
    df = pd.DataFrame(dic, index=['date'])

    return df

In [11]:
def parseText(src, remove):
    for r in remove: 
        src = src.replace(r,'')
    src = src.split(' ')
    if '' in src:
        src.remove('')
    src = [int(i) for i in src]
    return src

In [12]:
def parseResult(r):
    remove = ['+', "Bonus",',', '$']
    winningNum = parseText(r('.winning-numbers-container li').text(), remove)
    prize = parseText(r('#lotto-max-game1collapse .prize-breakdown h3').text(), [',', '$', 'IN CASH WON', '!'])
    #winners = r('#lotto-max-game1collapse .game-prize-3col-table tbody tr td.chart-winners-col').text()
    #fp_winners = parseText(r('#lotto-max-game1collapse .game-prize-2col-table tbody tr td.chart-winners-col').text(), remove)
    winners_count = parseText(r('#lotto-max-game1collapse .game-prize-3col-table tbody tr td.chart-odds-col').text(), remove)
    fp_winners_count = parseText(r('#lotto-max-game1collapse .game-prize-2col-table tbody tr td.chart-prize-col').text(), ['FREE PLAY winners','$', ','])
    total_winners_count = winners_count + fp_winners_count
    return winningNum, prize, total_winners_count

winning number=[] prize = [] total winners = []


IndexError: list index out of range

In [23]:
    url = generateURL(t)
    #urlopen(url).read().decode('ISO-8859-1')
    r = pq(url = url )

In [24]:
r.text()
r('.winning-numbers-container')

[]

In [22]:
endDate = dt.date(2009, 9, 19)
f = open("data.txt", "w")
try:
    while t > endDate:
        data = process(t)
        df.to_csv(f, mode='a', header=False)
        t = t - dt.timedelta(7)
        time.sleep(20)
except: 
    print(df)

winning number=[] prize = [] total winners = []
Empty DataFrame
Columns: []
Index: []


In [4]:
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

# Main function
def run_test():
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  # Choose a random proxy
  proxy_index = random_proxy()
  proxy = proxies[proxy_index]

  for n in range(1, 100):
    # Every 10 requests, generate a new proxy
    if n % 10 == 0:
        proxy_index = random_proxy()
        proxy = proxies[proxy_index]
    
    endDate = dt.date(2009, 9, 19)
    df = pd.DataFrame()
    # Make the call
    try:
        while t > endDate:
            req = Request(generateURL(t))
            req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')
            my_ip = urlopen(req).read().decode('utf8')
            data = process(t)
            df.append(data)
            t = t - dt.timedelta(7)
            print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
      del proxies[proxy_index]
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)